In [115]:
import pandas as pd
import json

In [123]:
apartments = pd.read_csv('../data/curated/geometry_property.csv')

In [116]:
tram_stops = pd.read_csv('../data/curated/tram_stop.csv')
train_stations = pd.read_csv('../data/curated/train_station.csv')
schools = pd.read_csv('../data/curated/school_locations.csv')
restaurants = pd.read_csv('../data/curated/melb_resto.csv')
landmarks = pd.read_csv('../data/curated/melb_landmarks.csv')

In [40]:
apartments.loc[apartments['Unnamed: 0'] == 1855]

,Unnamed: 0,Address,Suburb,Prices,Estimated resident population (no.),Population density (persons/km2),Median age - persons (years),Working age population (aged 15-64 years) (no.),Employed (no.),Unemployed (no.),...,Apartments - total (no.),Median weekly household rental payment ($),Rented (no.),Count of homeless persons (no.),"Used at least one form of public transport (train, tram, bus, ferry) (no.)",latitude,longitude,num_stops,num_stations,total_transport
1855,1855,"207B/58 Myrtle St, \nIVANHOE VIC 3079",IVANHOE,550.0,12561.0,2487.4,40.9,8357.0,0.0,0.0,...,1529.0,440.0,1505.0,16.0,343.0,NaN,NaN,0.0,0.0,0.0


In [35]:
train_stations

,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,TICKETZONE,ROUTEUSSP,geometry
0,19970,Royal Park Railway Station (Parkville),-37.781193,144.952301,1,Upfield,POINT (144.95230710206772 -37.781179806470384)
1,19971,Flemington Bridge Railway Station (North Melbo...,-37.788140,144.939323,1,Upfield,POINT (144.93932910356588 -37.78812681562748)
2,19972,Macaulay Railway Station (North Melbourne),-37.794267,144.936166,1,Upfield,POINT (144.9361719034498 -37.79425383545)
3,19973,North Melbourne Railway Station (West Melbourne),-37.807419,144.942570,1,"Flemington,Sunbury,Upfield,Werribee,Williamsto...",POINT (144.9425759409124 -37.80740580596864)
4,19974,Clifton Hill Railway Station (Clifton Hill),-37.788657,144.995417,1,"Mernda,Hurstbridge",POINT (144.99542285377925 -37.78864386946561)
...,...,...,...,...,...,...,...
215,19965,Coburg Railway Station (Coburg),-37.742345,144.963336,1,Upfield,POINT (144.9633418243668 -37.742331846983326)
216,19966,Moreland Railway Station (Coburg),-37.754485,144.961823,1,Upfield,POINT (144.96182910331285 -37.75447185695532)
217,19967,Anstey Railway Station (Brunswick),-37.761242,144.960684,1,Upfield,POINT (144.96069006277764 -37.761228876919425)
218,19968,Brunswick Railway Station (Brunswick),-37.767721,144.959587,1,Upfield,POINT (144.9595927822653 -37.767707796884636)


In [122]:
# Calculate Haversine Distance
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the earth in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat / 2) * math.sin(dLat / 2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c  # Distance in km
    return d

In [120]:
for index, row in apartments.iterrows():
    lon1 = row['longitude']
    lat1 = row['latitude']

    num_stops = 0 
    for index_stops, row2 in tram_stops.iterrows():
        lon2 = row2['LONGITUDE']
        lat2 = row2['LATITUDE']

        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 0.5:
            num_stops += 1

apartments.at[index, 'num_stops'] = num_stops

In [110]:
for index, row in apartments.iterrows():
    lon1 = row['longitude']
    lat1 = row['latitude']
    
    num_stations = 0
    for index_stations, row2 in train_stations.iterrows():
        lon2 = row2['LONGITUDE']
        lat2 = row2['LATITUDE']
    
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 0.5:
            num_stations += 1
    apartments.at[index, 'num_stations'] = num_stations

In [112]:
for index, row in apartments.iterrows():
    lon1 = row['longitude']
    lat1 = row['latitude']
    
    num_schools = 0
    for index_schools, row2 in schools.iterrows():
        lon2 = row2['X']
        lat2 = row2['Y']
    
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 2:
            num_schools += 1
    apartments.at[index, 'num_schools'] = num_schools

In [113]:
for index, row in apartments.iterrows():
    lon1 = row['longitude']
    lat1 = row['latitude']
    
    num_restaurants = 0
    for index_restaurants, row2 in restaurants.iterrows():
        lon2 = row2['longitude']
        lat2 = row2['latitude']
    
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 1:
            num_restaurants += 1
    apartments.at[index, 'num_restaurants'] = num_restaurants

In [117]:
# Preprocess the 'co_ordinates' column to ensure it's valid JSON
landmarks['co_ordinates'] = landmarks['co_ordinates'].str.replace("'", "\"")
landmarks['co_ordinates'] = landmarks['co_ordinates'].apply(json.loads)

landmarks['lon'] = landmarks['co_ordinates'].apply(lambda x:x['lon'])
landmarks['lat'] = landmarks['co_ordinates'].apply(lambda x:x['lat'])

In [118]:
for index, row in apartments.iterrows():
    lon1 = row['longitude']
    lat1 = row['latitude']
    
    num_landmarks = 0
    for index_landmarks, row2 in landmarks.iterrows():
        lon2 = row2['lon']
        lat2 = row2['lat']
    
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 1:
            num_landmarks += 1
    apartments.at[index, 'num_landmarks'] = num_landmarks

In [121]:
apartments['total_transport'] = apartments['num_stops'] + apartments['num_stations']
apartments

,Unnamed: 0,Address,Suburb,Prices,Estimated resident population (no.),Population density (persons/km2),Median age - persons (years),Working age population (aged 15-64 years) (no.),Employed (no.),Unemployed (no.),...,Count of homeless persons (no.),"Used at least one form of public transport (train, tram, bus, ferry) (no.)",latitude,longitude,num_landmarks,num_stops,num_stations,num_schools,num_restaurants,total_transport
0,0,"39 Durham Crescent, \nHOPPERS CROSSING VIC 3029",HOPPERS CROSSING,450.0,37666.0,4420.4,74.1,25272.0,0.0,0.0,...,228.0,884.0,-37.872563,144.686795,0.0,NaN,0.0,11.0,0.0,NaN
1,1,"7/1 Mabel Street, \nIVANHOE VIC 3079",IVANHOE,450.0,12561.0,2487.4,40.9,8357.0,0.0,0.0,...,16.0,343.0,-37.761896,145.029437,0.0,NaN,0.0,13.0,0.0,NaN
2,2,"2/59 Green Street, \nIVANHOE VIC 3079",IVANHOE,450.0,12561.0,2487.4,40.9,8357.0,0.0,0.0,...,16.0,343.0,-37.764732,145.035655,0.0,NaN,0.0,12.0,0.0,NaN
3,3,"104/15 Ivanhoe Parade, \nIVANHOE VIC 3079",IVANHOE,450.0,12561.0,2487.4,40.9,8357.0,0.0,0.0,...,16.0,343.0,-37.765456,145.044514,0.0,NaN,1.0,12.0,0.0,NaN
4,4,"63 Dunne Street, \nKINGSBURY VIC 3083",KINGSBURY,450.0,10564.0,1172.6,34.0,7808.0,0.0,0.0,...,120.0,382.0,-37.714056,145.037364,0.0,NaN,0.0,8.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4366,4366,"156 Black Forest Road, \nWYNDHAM VALE VIC 3024",WYNDHAM VALE,465.0,22858.0,2941.3,63.4,15428.0,0.0,0.0,...,101.0,651.0,-37.899498,144.609698,0.0,NaN,0.0,3.0,0.0,NaN
4367,4367,"104/118 Pier Street, \nALTONA VIC 3018",ALTONA,470.0,28738.0,1704.2,78.8,18016.0,0.0,0.0,...,108.0,600.0,-37.868826,144.830352,0.0,NaN,1.0,5.0,0.0,NaN
4368,4368,"2/35 Ailsa Street, \nALTONA MEADOWS VIC 3028",ALTONA MEADOWS,470.0,18665.0,1770.6,40.9,12311.0,0.0,0.0,...,52.0,380.0,-37.870238,144.762616,0.0,NaN,1.0,4.0,0.0,NaN
4369,4369,"6 Stringybark Court, \nBERWICK VIC 3806",BERWICK,470.0,23187.0,1174.1,43.3,14432.0,0.0,0.0,...,140.0,241.0,-38.026246,145.324107,0.0,NaN,0.0,0.0,0.0,NaN
